In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append('../../')

In [3]:
from datetime import datetime
from glob import glob
import os
import numpy as np
import pandas as pd

from tqdm import tqdm

from utils.db_util import create_postgres_engine, make_query
from utils.query_util import query_person_ids_in_organization
from utils.experience_processor import (
    parse_work_date, parse_work_duration, diff_month, get_person_stay_term,
    is_current_job
)

In [4]:
INPUT_DIR = '/home/sjb/Projects/Research/LinkedIn_OB/data/company_level_individual_profiles'
OUTPUT_DIR = '/home/sjb/Projects/Research/LinkedIn_OB/data/company_level_individual_stay_term'
OUTPUT_FILE_NAME_FORMAT = '{short_name}_person_stay_term.csv'

In [5]:
ORG_PROFILE_SUFFIX = '?trk=ppro_cprof'
ORG_PROFILE_PREFIX = '/company/'

In [6]:
db_conn = create_postgres_engine()

In [7]:
source_files = glob(os.path.join(INPUT_DIR,
                                 '*.csv'))

In [8]:
# sample = pd.read_csv(source_files[0])

In [9]:
# org_profile_link = sample['org_profile_link'].values[0]
# person_ids = sample['person_id'].tolist()

In [10]:
def escape_single_quote(text):
    """Since postgres does not like single quote in text"""
    text = text.replace("'", "''")
    return text

In [11]:
def apply_func_when_col_vals_not_null(func,
                                      r,
                                      arg_cols,
                                      cols,):
    check_null = r[cols]
    
    if check_null.isnull().any():
        return np.nan

    else:
        func_args = [r[c] for c in arg_cols]
        return func(*func_args)

In [12]:
def get_person_org_stay_term_df(org_profile_link,
                                person_ids,
                                save=True):
    org_profile_link_escaped = escape_single_quote(org_profile_link)
    org_person_df = query_person_ids_in_organization(db_conn, org_profile_link_escaped, person_ids)
    
    assert len(set(org_person_df['person_id']).symmetric_difference(set(person_ids))) == 0
    
    org_person_df['date_start_parsed'] = org_person_df['date_start'].apply(
    lambda x: parse_work_date(x, date_type='start_date'))

    org_person_df['date_end_parsed'] = org_person_df['date_end'].apply(
        lambda x: parse_work_date(x, date_type='end_date'))

    org_person_df['duration_parsed'] = org_person_df.apply(
        lambda r: apply_func_when_col_vals_not_null(
            parse_work_duration,
            r,
            ['duration'],
            ['date_start_parsed', 'date_end_parsed'],
        ), axis=1)
    
    person_stay_term_df = org_person_df.groupby('person_id').apply(get_person_stay_term).reset_index()
    person_stay_term_df.columns = ['person_id', 'n_months']
    
    min_person_stay_term = person_stay_term_df['n_months'].min() 
    if min_person_stay_term < 0:
        raise ValueError('min of person stay term is {}, please investigate'.format(
            min_person_stay_term))
        
    person_stay_term_df['is_current_job'] = person_stay_term_df['person_id'].apply(
        lambda x: is_current_job(x, org_person_df))
    
    assert len(set(person_stay_term_df['person_id']).symmetric_difference(set(person_ids))) == 0
    
    if save:
        short_link = org_profile_link.replace(ORG_PROFILE_PREFIX, '').replace(ORG_PROFILE_SUFFIX, '')
        output_file_path = os.path.join(OUTPUT_DIR,
                                        OUTPUT_FILE_NAME_FORMAT.format(
                                            short_name=short_link))
        person_stay_term_df.to_csv(output_file_path, index=False)

    
    return person_stay_term_df

In [13]:
for fp in tqdm(source_files):
    
    
    org_df = pd.read_csv(fp)
    org_profile_link = org_df['org_profile_link'].values[0]
    person_ids = org_df['person_id'].tolist()
    
    _ = get_person_org_stay_term_df(org_profile_link,
                                    person_ids,
                                    save=True)

  0%|          | 1/1428 [00:06<2:25:34,  6.12s/it]

Unexpected duration string format showing años


  0%|          | 5/1428 [00:39<2:32:02,  6.41s/it]

Unexpected duration string format showing ans
Unexpected duration string format showing an
Unexpected duration string format showing an
Unexpected duration string format showing Monate


  1%|          | 8/1428 [00:55<2:25:02,  6.13s/it]

...

 98%|█████████▊| 1398/1428 [4:47:13<02:45,  5.53s/it]

Unexpected duration string format showing als


 99%|█████████▉| 1412/1428 [4:49:22<03:41, 13.82s/it]

Unexpected duration string format showing an
Unexpected duration string format showing an


100%|██████████| 1428/1428 [4:55:13<00:00, 12.40s/it]


In [14]:
org_profile_link

'/company/target?trk=ppro_cprof'